In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import win32com.client
import time
import getpass
from datetime import timedelta

usuario = getpass.getuser()

inicio = time.time()
# %%
import os
from datetime import datetime


In [2]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Define the directory path
carpeta_fechas = "C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Actualización diaria fechas DT/Actualización Diaria fechas Dts R3"

# Maximum number of days to look back
max_days_back = 30

# Function to try reading the file for a given date
def try_read_file(date_str):
    ruta_arch = f'{date_str} Actualizacion fechas diaria  Dts OEM.xlsx'
    ruta = os.path.join(carpeta_fechas, ruta_arch)
    if os.path.exists(ruta):
        df_fechas = pd.read_excel(ruta,sheet_name='Data', dtype = {'Nro. DT':'str'})
        print(f"File found: {ruta}")
        return df_fechas
    return None

# Try to find a file for today and up to max_days_back days ago
for days_back in range(max_days_back + 1):
    date_to_try = (datetime.today() - timedelta(days=days_back)).strftime('%d-%m-%Y')
    df_fechas = try_read_file(date_to_try)
    if df_fechas is not None:
        break
else:
    print("No file found within the given date range.")

# Continue with your further processing
if df_fechas is not None:
    # Do something with df_fechas
    pass
else:
    # Handle the case where no file was found
    pass




No file found within the given date range.


In [16]:
df_fechas['Nro. DT'].to_clipboard(index = False, header = False)

In [17]:
df_fechas['Nro. DT'].nunique()

1858

In [18]:
try:
    # Initialize SAP GUI scripting
    SapGuiAuto = win32com.client.GetObject("SAPGUI")
    application = SapGuiAuto.GetScriptingEngine
    if not application:
        raise Exception("Error obtaining SAP GUI application")

    connection = application.Children(0)
    if not connection:
        raise Exception("Error obtaining SAP GUI connection")

    session = connection.Children(0)
    if not session:
        raise Exception("Error obtaining SAP GUI session")

    # Connect to WScript if available
    try:
        WScript = win32com.client.Dispatch("WScript")
        WScript.ConnectObject(session, "on")
        WScript.ConnectObject(application, "on")
    except Exception as e:
        print(f"Error connecting to WScript: {str(e)}")

    # Maximize the window
    session.findById("wnd[0]").maximize()

    # Execute transaction
    session.findById("wnd[0]/tbar[0]/okcd").text = "zmm_seguim_comex_cl"
    session.findById("wnd[0]").sendVKey(0)

    # Press buttons
    session.findById("wnd[0]/usr/btn%_P_TKNUM_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[0]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]").sendVKey(8)

    # Export to Excel
    session.findById("wnd[0]/usr/cntlALV_COMEX/shellcont/shell").pressToolbarContextButton("&MB_EXPORT")
    session.findById("wnd[0]/usr/cntlALV_COMEX/shellcont/shell").selectContextMenuItem("&XXL")
    session.findById("wnd[1]/tbar[0]/btn[0]").press()

    # Set file path and name
    session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Codigos\\automatizacion_gere_comex"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").text = "comex.XLSX"
    session.findById("wnd[1]/usr/ctxtDY_FILENAME").caretPosition = 5
    session.findById("wnd[1]/tbar[0]/btn[11]").press()

    # Close SAP windows
    session.findById("wnd[0]").sendVKey(3)
    session.findById("wnd[0]").sendVKey(3)

except Exception as e:
    print(f"Error during SAP GUI interaction: {str(e)}")






Error connecting to WScript: (-2147221005, 'Cadena clase no válida', None, None)


In [19]:
# %%
import xlwings as xw
try:
    book = xw.Book("C:/Users/lravlic/Codigos/automatizacion_gere_comex/comex.XLSX")
    book.close()
except Exception as e:
    print(e)


In [20]:
comex = pd.read_excel("C:/Users/lravlic/Codigos/automatizacion_gere_comex/comex.XLSX")

In [21]:
dts_a_mir6 = comex[comex['Estatus']=='CERRADO']

In [22]:
dts_a_mir6.Estatus.value_counts()

Estatus
CERRADO    1730
Name: count, dtype: int64

In [23]:
dts_a_mir6['Nro. DT asociado'] = dts_a_mir6['Nro. DT asociado'].astype(str)

# Create the new column 'DT' by concatenating '*' before and after the values
dts_a_mir6['DT'] = '*' + dts_a_mir6['Nro. DT asociado'] + '*'

C:\Users\lravlic\AppData\Local\Temp\ipykernel_31920\1582769832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dts_a_mir6['Nro. DT asociado'] = dts_a_mir6['Nro. DT asociado'].astype(str)
C:\Users\lravlic\AppData\Local\Temp\ipykernel_31920\1582769832.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dts_a_mir6['DT'] = '*' + dts_a_mir6['Nro. DT asociado'] + '*'


In [24]:
dts_a_mir6['DT'].to_clipboard(index=False,header=False)

In [25]:
import win32com.client

# Initialize SAP GUI scripting
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Enter transaction code "mir6"
session.findById("wnd[0]/tbar[0]/okcd").text = "mir6"
session.findById("wnd[0]").sendVKey(0)

# Clear and set the focus on the user field
session.findById("wnd[0]/usr/txtSO_USNAM-LOW").text = ""
session.findById("wnd[0]/usr/txtSO_USNAM-LOW").setFocus()
session.findById("wnd[0]/usr/txtSO_USNAM-LOW").caretPosition = 0
session.findById("wnd[0]").sendVKey(0)

# Set focus on the text field and perform actions
session.findById("wnd[0]/usr/txtSO_BKTXT-LOW").setFocus()
session.findById("wnd[0]/usr/txtSO_BKTXT-LOW").caretPosition = 0
session.findById("wnd[0]/usr/btn%_SO_BKTXT_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Toggle checkboxes
session.findById("wnd[0]/usr/chkP_IV_BG").setFocus()
session.findById("wnd[0]/usr/chkP_IV_BG").selected = False
session.findById("wnd[0]").sendVKey(2)
session.findById("wnd[0]/usr/chkP_IV_BG").selected = True
session.findById("wnd[0]/usr/chkP_IV_IS").setFocus()
session.findById("wnd[0]/usr/chkP_IV_IS").selected = True
session.findById("wnd[0]/usr/chkP_IV_OV").setFocus()
session.findById("wnd[0]/usr/chkP_IV_OV").selected = True
session.findById("wnd[0]/usr/chkP_IV_STO").setFocus()
session.findById("wnd[0]/usr/chkP_IV_STO").selected = True
session.findById("wnd[0]/usr/chkP_IV_EDI").setFocus()
session.findById("wnd[0]/usr/chkP_IV_EDI").selected = True
session.findById("wnd[0]/usr/chkP_IV_RAP").setFocus()
session.findById("wnd[0]/usr/chkP_IV_RAP").selected = True
session.findById("wnd[0]/usr/chkP_IV_BAP").setFocus()
session.findById("wnd[0]/usr/chkP_IV_BAP").selected = True
session.findById("wnd[0]/usr/chkP_IV_PAR").setFocus()
session.findById("wnd[0]/usr/chkP_IV_PAR").selected = True
session.findById("wnd[0]/usr/chkP_IV_ERS").setFocus()
session.findById("wnd[0]/usr/chkP_IV_ERS").selected = True
session.findById("wnd[0]/usr/chkP_IV_SRM").setFocus()
session.findById("wnd[0]/usr/chkP_IV_SRM").selected = True
session.findById("wnd[0]/usr/chkP_IV_TP").setFocus()
session.findById("wnd[0]/usr/chkP_IV_TP").selected = True
session.findById("wnd[0]/usr/chkP_IV_A2A").setFocus()
session.findById("wnd[0]/usr/chkP_IV_A2A").selected = True
session.findById("wnd[0]/usr/chkP_IV_B2B").setFocus()
session.findById("wnd[0]/usr/chkP_IV_B2B").selected = True

# Execute commands and navigate
session.findById("wnd[0]").sendVKey(8)
session.findById("wnd[0]/tbar[0]/btn[86]").press()
session.findById("wnd[0]/tbar[1]/btn[33]").press()
session.findById("wnd[1]/usr/lbl[1,6]").setFocus()
session.findById("wnd[1]/usr/lbl[1,6]").caretPosition = 7
session.findById("wnd[1]").sendVKey(2)
session.findById("wnd[0]/tbar[1]/btn[43]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path for export
session.findById("wnd[1]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Codigos\\land_cost"
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 34
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)


In [26]:

try:
    book = xw.Book("C:/Users/lravlic/Codigos/land_cost/export.XLSX")
    book.close()
except Exception as e:
    print(e)


In [27]:
df_mir6 = pd.read_excel("C:/Users/lravlic/Codigos/land_cost/export.XLSX")

In [28]:
df_mir6_filtro_yr = df_mir6[df_mir6['Fe.contabilización'].dt.year >=2024]

In [29]:
df_mir6_filtro_yr['Texto cab.documento'] = pd.to_numeric(df_mir6_filtro_yr['Texto cab.documento'], errors='coerce')

# Convert the column to integers, dropping NaN values first
df_mir6_filtro_yr = df_mir6_filtro_yr.dropna(subset=['Texto cab.documento'])
df_mir6_filtro_yr['Texto cab.documento'] = df_mir6_filtro_yr['Texto cab.documento'].astype(int)

# Display the DataFrame to verify the changes
print(df_mir6_filtro_yr['Texto cab.documento'])

172      37442
173      19551
174      19555
175      19660
176      19760
         ...  
30725    39899
30726    39897
30727    39896
30728    39897
30729    39896
Name: Texto cab.documento, Length: 8342, dtype: int32


C:\Users\lravlic\AppData\Local\Temp\ipykernel_31920\2222300859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mir6_filtro_yr['Texto cab.documento'] = pd.to_numeric(df_mir6_filtro_yr['Texto cab.documento'], errors='coerce')


In [30]:
df_mir6_filtro_yr['Texto cab.documento'] = df_mir6_filtro_yr['Texto cab.documento'].astype('str')

In [31]:
filtered_df = df_mir6_filtro_yr[df_mir6_filtro_yr['Texto cab.documento'].str.len() <= 6]

# Print the filtered DataFrame
print(filtered_df['Texto cab.documento'])


172      37442
173      19551
174      19555
175      19660
176      19760
         ...  
30725    39899
30726    39897
30727    39896
30728    39897
30729    39896
Name: Texto cab.documento, Length: 8257, dtype: object


In [32]:
filtered_df.columns

Index(['Tipo de cambio', 'Texto cab.documento', 'Referencia',
       'Nombre del usuario', 'Nombre 1', 'Nº documento factura', 'Moneda',
       'Impte.bruto factura', 'Fecha de entrada', 'Fecha de documento',
       'Fe.descto.', 'Fe.contabilización', 'Emisor de factura', 'Ejercicio',
       'Creado por', 'Código transacción', 'Clase de documento',
       'Base retención', 'Asignación', 'Anulado con'],
      dtype='object')

In [33]:
filtered_df['Referencia'].to_clipboard(header=False, index=False)
# Initialize SAP GUI scripting
SapGuiAuto = win32com.client.GetObject("SAPGUI")
application = SapGuiAuto.GetScriptingEngine
connection = application.Children(0)
session = connection.Children(0)

# Maximize the window
session.findById("wnd[0]").maximize()

# Execute transaction code "fbl3n"
session.findById("wnd[0]/tbar[0]/okcd").text = "fbl3n"
session.findById("wnd[0]").sendVKey(0)

# Interact with elements
session.findById("wnd[0]/tbar[1]/btn[16]").press()
session.findById("wnd[0]/usr/ssub%_SUBSCREEN_%_SUB%_CONTAINER:SAPLSSEL:2001/ssubSUBSCREEN_CONTAINER2:SAPLSSEL:2000/cntlSUB_CONTAINER/shellcont/shellcont/shell/shellcont[1]/shell").selectNode("         37")
session.findById("wnd[0]/usr/ssub%_SUBSCREEN_%_SUB%_CONTAINER:SAPLSSEL:2001/ssubSUBSCREEN_CONTAINER2:SAPLSSEL:2000/cntlSUB_CONTAINER/shellcont/shellcont/shell/shellcont[1]/shell").topNode = "         30"
session.findById("wnd[0]/usr/ssub%_SUBSCREEN_%_SUB%_CONTAINER:SAPLSSEL:2001/ssubSUBSCREEN_CONTAINER2:SAPLSSEL:2000/cntlSUB_CONTAINER/shellcont/shellcont/shell/shellcont[1]/shell").doubleClickNode("         37")
session.findById("wnd[0]/usr/ssub%_SUBSCREEN_%_SUB%_CONTAINER:SAPLSSEL:2001/ssubSUBSCREEN_CONTAINER2:SAPLSSEL:2000/ssubSUBSCREEN_CONTAINER:SAPLSSEL:1106/btn%_%%DYN009_%_APP_%-VALU_PUSH").press()
session.findById("wnd[1]/tbar[0]/btn[24]").press()
session.findById("wnd[1]/tbar[0]/btn[0]").press()
session.findById("wnd[1]/tbar[0]/btn[8]").press()

# Set selection criteria
session.findById("wnd[0]/usr/radX_AISEL").select()
session.findById("wnd[0]/usr/ctxtSD_SAKNR-LOW").text = "1104040001"
session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").text = "CL07"
session.findById("wnd[0]/usr/ctxtSO_BUDAT-LOW").text = "01.01.2024"
session.findById("wnd[0]/usr/ctxtSO_BUDAT-HIGH").text = "31.12.2024"
session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").caretPosition = 4
session.findById("wnd[0]").sendVKey(8)

# Change selection criteria

session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").text = "CL02"
session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").setFocus()
session.findById("wnd[0]/usr/ctxtSD_BUKRS-LOW").caretPosition = 4
session.findById("wnd[0]").sendVKey(8)

# Execute commands and navigate
session.findById("wnd[0]/usr/lbl[29,2]").setFocus()
session.findById("wnd[0]/usr/lbl[29,2]").caretPosition = 69
session.findById("wnd[0]").sendVKey(16)
session.findById("wnd[1]/tbar[0]/btn[0]").press()

# Set the file path for export
session.findById("wnd[1]/usr/ctxtDY_PATH").setFocus()
session.findById("wnd[1]/usr/ctxtDY_PATH").caretPosition = 0
session.findById("wnd[1]").sendVKey(4)
session.findById("wnd[2]/usr/ctxtDY_PATH").text = "C:\\Users\\lravlic\\Codigos\\land_cost"
session.findById("wnd[2]/usr/ctxtDY_FILENAME").text = "iva_r3.XLSX"
session.findById("wnd[2]/usr/ctxtDY_FILENAME").caretPosition = 6
session.findById("wnd[2]/tbar[0]/btn[11]").press()
session.findById("wnd[1]/tbar[0]/btn[11]").press()

# Close the windows
session.findById("wnd[0]").sendVKey(3)
session.findById("wnd[0]").sendVKey(3)

try:
    book = xw.Book("C:/Users/lravlic/Codigos/land_cost/iva_r3.XLSX")
    book.close()
except Exception as e:
    print(e)
df_iva = pd.read_excel('iva_r3.xlsx')
df_iva=df_iva[['Referencia','Importe en moneda local']]
df_iva = df_iva.groupby('Referencia').max().reset_index()

filtered_df = filtered_df.merge(df_iva, left_on='Referencia',right_on='Referencia', how='left')
filtered_df['Importe en moneda local'].fillna(0, inplace=True)
filtered_df['Impte.bruto factura'] = filtered_df['Impte.bruto factura'] - filtered_df['Importe en moneda local']
filtered_df.drop(columns='Importe en moneda local', inplace=True)


In [34]:
filtered_df.head()

,Tipo de cambio,Texto cab.documento,Referencia,Nombre del usuario,Nombre 1,Nº documento factura,Moneda,Impte.bruto factura,Fecha de entrada,Fecha de documento,Fe.descto.,Fe.contabilización,Emisor de factura,Ejercicio,Creado por,Código transacción,Clase de documento,Base retención,Asignación,Anulado con
0,"1,00000",37442,3951095983-7,PPINTOP,NaN,5112950611,CLP,19510382.0,2024-01-10,2024-01-08,2024-01-08,2024-01-10,10000788,2024,NaN,MIRO,KI,0,3951095983-7,NaN
1,"1,00000",19551,0236960581,SROJASF,NaN,5112952310,CLP,0.0,2024-01-11,2024-01-09,2024-01-09,2024-01-11,10000788,2024,NaN,MIRO,KI,0,0236960581,NaN
2,"1,00000",19555,0236960587,SROJASF,NaN,5112952457,CLP,0.0,2024-01-11,2024-01-09,2024-01-09,2024-01-11,10000788,2024,NaN,MIRO,KI,0,0236960587,NaN
3,"1,00000",19660,0236966448,SROJASF,NaN,5112956420,CLP,0.0,2024-01-16,2024-01-13,2024-01-13,2024-01-16,10000788,2024,NaN,MIRO,KI,0,0236966448,NaN
4,"1,00000",19760,0236965402,SROJASF,NaN,5112956464,CLP,25314.0,2024-01-16,2024-01-12,2024-01-12,2024-01-16,10000788,2024,NaN,MIRO,KI,0,0236965402,NaN


In [35]:
filtered_df.to_parquet('base_landcost.parquet')

In [36]:
glosario = pd.read_csv('glosario.csv', sep=';', usecols=['CLASE', 'Denominación'])

In [37]:
tipo_gasto = pd.read_csv('tipo_de_gasto.csv', sep=';')

In [38]:
tipo_gasto.dropna(subset='R3', inplace=True)

In [39]:
tipo_gasto.drop_duplicates(subset='R3', inplace=True, keep='first')

In [40]:
filtered_df = filtered_df.merge(tipo_gasto, left_on='Emisor de factura', right_on='R3', how='left')

In [41]:
filtered_df[filtered_df['Tipo de gasto'].isna()]

,Tipo de cambio,Texto cab.documento,Referencia,Nombre del usuario,Nombre 1,Nº documento factura,Moneda,Impte.bruto factura,Fecha de entrada,Fecha de documento,...,Ejercicio,Creado por,Código transacción,Clase de documento,Base retención,Asignación,Anulado con,Mar,Tipo de gasto,R3
5569,"918,86000",35103,278491723,SROJASF,NaN,5113307639,USD,3788.00,2024-08-22,2024-07-11,...,2024,NaN,MIRO,KX,0,278491723,NaN,NaN,NaN,NaN
5570,"918,86000",35192,278491737,JTOLOZA,NaN,5113308479,USD,3788.00,2024-08-22,2024-07-11,...,2024,NaN,MIRO,KX,0,278491737,NaN,NaN,NaN,NaN
5571,"908,92000",36337,241838676,JTOLOZA,NaN,5113316735,USD,3788.00,2024-08-27,2024-07-12,...,2024,NaN,MIRO,KX,0,241838676,NaN,NaN,NaN,NaN
5572,"940,15000",35504,278491748,JTOLOZA,NaN,5113339405,USD,2788.00,2024-09-09,2024-07-19,...,2024,NaN,MIRO,KX,0,278491748,NaN,NaN,NaN,NaN
5573,"910,79000",35461,241472065,SROJASF,NaN,5113347150,USD,3938.00,2024-09-16,2024-07-17,...,2024,NaN,MIRO,KX,0,241472065,NaN,NaN,NaN,NaN
5574,"940,15000",35260,241472346,SROJASF,NaN,5113347432,USD,1921.00,2024-09-16,2024-07-19,...,2024,NaN,MIRO,KX,0,241472346,NaN,NaN,NaN,NaN
5575,"957,57000",37268,242077239,SROJASF,NaN,5113368341,USD,3183.67,2024-10-01,2024-08-06,...,2024,NaN,MIRO,KX,0,242077239,NaN,NaN,NaN,NaN
5576,"957,57000",37152,242494114,SROJASF,NaN,5113368342,USD,2183.67,2024-10-01,2024-08-06,...,2024,NaN,MIRO,KX,0,242494114,NaN,NaN,NaN,NaN
5577,"932,62000",36941,242262103,JTOLOZA,NaN,5113371615,USD,2138.00,2024-10-03,2024-08-12,...,2024,NaN,MIRO,KX,0,242262103,NaN,NaN,NaN,NaN
5578,"932,28000",37555,278491788,SROJASF,NaN,5113371987,USD,1788.00,2024-10-03,2024-08-15,...,2024,NaN,MIRO,KX,0,278491788,NaN,NaN,NaN,NaN


In [42]:
filtered_df.drop(columns=['Mar', 'R3'], inplace=True)

In [43]:
df_prov = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras Maestros/Maestro Proveedores/Base Proveedor OFICIAL OEM AXS.xlsx")

In [44]:
df_prov = df_prov[['Nombre de Proveedor','Equipo','Sector','Analista','Proveedor R3']]

In [45]:
df_fechas.columns

Index(['Creado el', 'Nro. DT', 'Marca/Producto', 'Proveedor',
       'Nombre Proveedor', 'País Origen', 'Moneda', 'Ref. Prefijo Emb.', 'Vía',
       'Vía (Texto)', 'Referencia', 'Documento de embarque', 'Cant. Factura',
       'Valor Fact.', 'Embarcador', 'Nombre del Embarcador',
       'Nave / Aerolínea', 'Voyage / Vuelo', 'Peso Bruto total', 'Volumen',
       'Pto. Origen', 'Pto. Destino', 'Nro. Cert. Origen', 'Nro. Cert. Seguro',
       'Estatus COMEX', 'Fe. INV', 'TF de INV', 'Fe. ATD', 'TF de ATD',
       'Fe. ATA', 'TF de ATA', 'Fe. Pago Aduana', 'TF Pago Aduana',
       'Fe. Alm. Logístico', 'TF Alm. Logístico', 'Fe. Fin Recep.Emb',
       'TF de Fin Recep. Emb', 'Fe. Centro Distribución',
       'TF. Centro Distribución', 'Fe. MIGO', 'TF de MIGO', 'Fe. Disp.últ.SKU',
       'TF.Disp.últ.SKU', 'OC-INV (R)', 'INV-ATD (R)', 'ATD-ATA (R)',
       'ATA-COMEX (R)', 'COMEX-ALM LOG (R)', 'LT Total (R)',
       'Fe. Entrega a Comex', 'ALM LOG - CD  (R)', 'CD - MIGO (R)',
       'MIGO - 

In [46]:
df_fechas.drop_duplicates(subset='Nro. DT', keep='first', inplace=True)

In [47]:
df_fechas['Nro. DT'] = df_fechas['Nro. DT'].astype('str')

In [48]:
df_prov.drop_duplicates(subset='Proveedor R3', keep='first', inplace=True)

In [49]:
filtered_df = filtered_df.merge(df_prov, left_on='Emisor de factura', right_on='Proveedor R3', how='left')
filtered_df = filtered_df.merge(df_fechas, left_on='Texto cab.documento', right_on='Nro. DT', how='left')

In [50]:
filtered_df = filtered_df.merge(df_fechas, left_on='Texto cab.documento', right_on='Nro. DT', how='left')

In [52]:
columnas = ['Nro. DT','Proveedor','Nombre del Embarcador','Vía (Texto)', 'País Origen','Pto. Origen', 'Fe. ATA']
df_fechas_columnas = df_fechas[columnas]

In [53]:
df_fechas_columnas['Nro. DT'] = df_fechas_columnas['Nro. DT'].astype('str')

C:\Users\lravlic\AppData\Local\Temp\ipykernel_31920\4089450802.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fechas_columnas['Nro. DT'] = df_fechas_columnas['Nro. DT'].astype('str')


In [54]:
df_fechas_columnas.to_parquet('df_dts.parquet')

In [ ]:
# df_cruce_fechas = df_mir6_filtro_yr.merge(df_fechas_columnas, left_on='Texto cab.documento',right_on='Nro. DT', how='left')

In [ ]:
final = time.time()

In [ ]:
time_difference = final - inicio

# Calculate minutes and seconds
minutes, seconds = divmod(time_difference, 60)

print(f"{minutes} minutos y {seconds} segundos")

2.0 minutos y 54.99405097961426 segundos


In [55]:
df_lc = pd.read_parquet('base_landcost.parquet')

In [ ]:
df_lc.head()

,Tipo de cambio,Texto cab.documento,Referencia,Nombre del usuario,Nombre 1,Nº documento factura,Moneda,Impte.bruto factura,Fecha de entrada,Fecha de documento,Fe.descto.,Fe.contabilización,Emisor de factura,Ejercicio,Creado por,Código transacción,Clase de documento,Base retención,Asignación,Anulado con
0,"1,00000",19551,0236960581,SROJASF,NaN,5112952310,CLP,0.0,2024-01-11,2024-01-09,2024-01-09,2024-01-11,10000788,2024,NaN,MIRO,KI,0,0236960581,NaN
1,"1,00000",19555,0236960587,SROJASF,NaN,5112952457,CLP,0.0,2024-01-11,2024-01-09,2024-01-09,2024-01-11,10000788,2024,NaN,MIRO,KI,0,0236960587,NaN
2,"1,00000",19660,0236966448,SROJASF,NaN,5112956420,CLP,0.0,2024-01-16,2024-01-13,2024-01-13,2024-01-16,10000788,2024,NaN,MIRO,KI,0,0236966448,NaN
3,"1,00000",19760,0236965402,SROJASF,NaN,5112956464,CLP,25314.0,2024-01-16,2024-01-12,2024-01-12,2024-01-16,10000788,2024,NaN,MIRO,KI,0,0236965402,NaN
4,"1,00000",19663,0236969029,SROJASF,NaN,5112956505,CLP,63816.0,2024-01-16,2024-01-15,2024-01-15,2024-01-16,10000788,2024,NaN,MIRO,KI,0,0236969029,NaN


In [56]:
antiguo = pd.read_excel("C:/Users/lravlic/Inchcape/Planificación y Compras Chile - Documentos/Planificación y Compras KPI-Reportes/Reporteria/LAND COST/LAND COST 2024 JUNIO - OEM.xlsx", sheet_name = 'BD Final')